## About
1. Convnext base trained with 768 image size and 1000 img size ensemble
2. Coatnet model using 2 stage training please refer attached notebooks for details
3. Dino Giant and Dino base
4. Tried using Lgbm instead of Lasso
5. Added Convnext tiny with 512 Image size

Credits : 
1. https://www.kaggle.com/code/takahitomizunobyts/lb-0-59-csiro-infer
2. https://www.kaggle.com/code/none00000/lb-0-57-infer-model-code
3. https://www.kaggle.com/code/mks2192/csiro-image2biomass-lb-59

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import cv2
from tqdm import tqdm
import gc

# ===============================================================
# 1. ⚙️ CONFIGURATION (PHẢI GIỐNG HỆT FILE TRAINING)
# ===============================================================
class CFG:
    # --- Đường dẫn (Paths) ---
    # (Hãy điều chỉnh các đường dẫn này cho đúng với môi trường của bạn)
    BASE_PATH = '/kaggle/input/csiro-biomass'
    TEST_CSV = os.path.join(BASE_PATH, 'test.csv')
    TEST_IMAGE_DIR = os.path.join(BASE_PATH, 'test')
    
    # Thư mục chứa 5 file .pth
    MODEL_DIR = '/kaggle/input/csiro-convnext-base-768-imsize-0-6436/' # Giả sử 5 file .pth nằm cùng thư mục
    SUBMISSION_FILE = 'submission.csv'
    
    # --- Cài đặt Mô hình (PHẢI TRÙNG KHỚP) ---
    MODEL_NAME = 'convnext_base' # PHẢI GIỐNG HỆT LÚC TRAIN
    IMG_SIZE = 768               # PHẢI GIỐNG HỆT LÚC TRAIN
    
    # --- Cài đặt Inference ---
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 1 # Có thể tăng batch size khi inference
    NUM_WORKERS = 1
    N_FOLDS = 5
    
    # --- Mục tiêu & Loss (PHẢI TRÙNG KHỚP) ---
    # 3 mục tiêu model đã dự đoán
    TARGET_COLS = ['Dry_Total_g', 'GDM_g', 'Dry_Green_g']
    
    # 5 mục tiêu để nộp bài
    ALL_TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']

print(f"Sử dụng thiết bị: {CFG.DEVICE}")
print(f"Backbone mô hình: {CFG.MODEL_NAME}")
print(f"Kích thước ảnh inference: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}")


# ===============================================================
# 2. 🏞️ AUGMENTATIONS (CHỈ DÙNG VALIDATION)
# ===============================================================
from albumentations import (
    Compose, 
    Resize, 
    Normalize,
    HorizontalFlip, 
    VerticalFlip
)

def get_tta_transforms():
    """
    Trả về một LIST các pipeline transform cho TTA.
    Mỗi pipeline là một "view" khác nhau của ảnh.
    """
    
    # Đây là các bước chuẩn hóa cơ bản
    base_transforms = [
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        ToTensorV2()
    ]
    
    # -----------------
    # View 1: Ảnh gốc (Chỉ Resize + Normalize)
    # -----------------
    original_view = Compose([
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    # -----------------
    # View 2: Lật ngang (HFlip)
    # -----------------
    hflip_view = Compose([
        HorizontalFlip(p=1.0), # Luôn luôn lật
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    # -----------------
    # View 3: Lật dọc (VFlip)
    # -----------------
    vflip_view = Compose([
        VerticalFlip(p=1.0), # Luôn luôn lật
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    return [original_view, hflip_view, vflip_view]

print("Đã định nghĩa hàm get_tta_transforms().")


class TestBiomassDataset(Dataset):
    """
    Dataset tùy chỉnh cho ảnh test (Chiến lược "Hai luồng").
    Sửa đổi để chấp nhận một pipeline transform cụ thể cho TTA.
    """
    def __init__(self, df, transform_pipeline, image_dir):
        self.df = df
        # (SỬA ĐỔI) Chấp nhận một pipeline đã được khởi tạo
        self.transforms = transform_pipeline 
        self.image_dir = image_dir
        self.image_paths = df['image_path'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. Lấy thông tin
        img_path_suffix = self.image_paths[idx]
        
        # 2. Đọc ảnh gốc (2000x1000)
        filename = os.path.basename(img_path_suffix)
        full_path = os.path.join(self.image_dir, filename)
        
        image = cv2.imread(full_path)
        if image is None:
            print(f"Warning: Không thể đọc ảnh: {full_path}. Trả về ảnh đen.")
            image = np.zeros((1000, 2000, 3), dtype=np.uint8)
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 3. Cắt (Crop) thành 2 ảnh (Trái và Phải)
        height, width, _ = image.shape
        mid_point = width // 2
        img_left = image[:, :mid_point]
        img_right = image[:, mid_point:]
        
        # 4. Áp dụng TTA Transform (CÙNG MỘT TRANSFORM cho cả 2)
        # (Ví dụ: Cả 2 ảnh đều bị lật ngang)
        img_left_tensor = self.transforms(image=img_left)['image']
        img_right_tensor = self.transforms(image=img_right)['image']
        
        # 5. Trả về
        return img_left_tensor, img_right_tensor

# ===============================================================
# 4. 🧠 MODEL ARCHITECTURE (SAO CHÉP TỪ FILE TRAIN)
# ===============================================================
class BiomassModel(nn.Module):
    """
    Kiến trúc mô hình (Hai luồng, Ba đầu ra)
    PHẢI GIỐNG HỆT file training.
    """
    def __init__(self, model_name, pretrained, n_targets=3):
        super(BiomassModel, self).__init__()
        
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained, # Sẽ là False khi inference
            num_classes=0,
            global_pool='avg'
        )
        
        self.n_features = self.backbone.num_features
        self.n_combined_features = self.n_features * 2
        
        # --- Đầu cho Dry_Total_g ---
        self.head_total = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )
        
        # --- Đầu cho GDM_g ---
        self.head_gdm = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )
        
        # --- Đầu cho Dry_Green_g ---
        self.head_green = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )

    def forward(self, img_left, img_right):
        features_left = self.backbone(img_left)
        features_right = self.backbone(img_right)
        combined = torch.cat([features_left, features_right], dim=1)
        
        out_total = self.head_total(combined)
        out_gdm = self.head_gdm(combined)
        out_green = self.head_green(combined)
        
        return out_total, out_gdm, out_green


def predict_one_view(models_list, test_loader, device):
    """
    Hàm con: Chạy dự đoán ensemble 5-fold cho MỘT view TTA.
    """
    view_preds_3 = {'total': [], 'gdm': [], 'green': []}
    
    with torch.no_grad():
        for (img_left, img_right) in tqdm(test_loader, desc="  Predicting View", leave=False):
            img_left = img_left.to(device)
            img_right = img_right.to(device)
            
            batch_preds_3_folds = {'total': [], 'gdm': [], 'green': []}
            
            # 1. Vòng lặp Ensemble 5-Fold
            for model in models_list:
                pred_total, pred_gdm, pred_green = model(img_left, img_right)
                batch_preds_3_folds['total'].append(pred_total.cpu())
                batch_preds_3_folds['gdm'].append(pred_gdm.cpu())
                batch_preds_3_folds['green'].append(pred_green.cpu())
            
            # 2. Lấy trung bình 5 Fold
            avg_pred_total = torch.mean(torch.stack(batch_preds_3_folds['total']), dim=0)
            avg_pred_gdm = torch.mean(torch.stack(batch_preds_3_folds['gdm']), dim=0)
            avg_pred_green = torch.mean(torch.stack(batch_preds_3_folds['green']), dim=0)
            
            view_preds_3['total'].append(avg_pred_total.numpy())
            view_preds_3['gdm'].append(avg_pred_gdm.numpy())
            view_preds_3['green'].append(avg_pred_green.numpy())

    # 3. Ghép kết quả các batch của view này
    preds_np = {
        'total': np.concatenate(view_preds_3['total']).flatten(),
        'gdm':   np.concatenate(view_preds_3['gdm']).flatten(),
        'green': np.concatenate(view_preds_3['green']).flatten()
    }
    return preds_np


def run_inference_with_tta():
    """
    Hàm inference chính, thực hiện TTA x Ensemble.
    """
    print(f"\n{'='*50}")
    print(f"🚀 BẮT ĐẦU INFERENCE (với TTA) 🚀")
    print(f"{'='*50}")

    # --- 1. Tải Dữ liệu Test ---
    print(f"Đang tải {CFG.TEST_CSV}...")
    try:
        test_df_long = pd.read_csv(CFG.TEST_CSV)
        test_df_unique = test_df_long.drop_duplicates(subset=['image_path']).reset_index(drop=True)
        print(f"Tìm thấy {len(test_df_unique)} ảnh test duy nhất.")
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy {CFG.TEST_CSV}")
        return None, None, None

    # --- 2. Tải 5 Mô hình (Ensemble) ---
    print("\nĐang tải 5 mô hình đã huấn luyện...")
    models_list = []
    # (Code tải 5 mô hình... giống hệt bước 16 của file trước)
    for fold in range(CFG.N_FOLDS):
        model_path = os.path.join(CFG.MODEL_DIR, f'best_model_fold{fold}.pth')
        if not os.path.exists(model_path):
            print(f"LỖI: Không tìm thấy file mô hình: {model_path}")
            return None, None, None
        model = BiomassModel(CFG.MODEL_NAME, pretrained=False)
        try:
            model.load_state_dict(torch.load(model_path, map_location=CFG.DEVICE))
        except RuntimeError:
            state_dict = torch.load(model_path, map_location=CFG.DEVICE)
            from collections import OrderedDict
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k.replace('module.', '')
                new_state_dict[name] = v
            model.load_state_dict(new_state_dict)
        model.eval()
        model.to(CFG.DEVICE)
        models_list.append(model)
    print(f"✓ Đã tải thành công {len(models_list)} mô hình.")

    # --- 3. Vòng lặp TTA (Vòng lặp ngoài) ---
    tta_transforms = get_tta_transforms()
    print(f"\nBắt đầu dự đoán với {len(tta_transforms)} TTA views...")
    
    all_tta_view_preds = [] # List để lưu kết quả của mỗi view TTA

    for i, tta_transform in enumerate(tta_transforms):
        print(f"--- Đang chạy TTA View {i+1}/{len(tta_transforms)} ---")
        
        # Tạo Dataset/Loader MỚI cho view TTA này
        test_dataset = TestBiomassDataset(
            df=test_df_unique,
            transform_pipeline=tta_transform, # Truyền pipeline TTA
            image_dir=CFG.TEST_IMAGE_DIR
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=CFG.BATCH_SIZE,
            shuffle=False,
            num_workers=CFG.NUM_WORKERS,
            pin_memory=True
        )
        
        # Chạy ensemble 5-fold cho view này
        view_preds_np = predict_one_view(models_list, test_loader, CFG.DEVICE)
        all_tta_view_preds.append(view_preds_np)
        print(f"✓ Hoàn thành TTA View {i+1}")

    # --- 4. Ensemble (Lấy trung bình) kết quả TTA ---
    print("\nĐang ensemble kết quả của các TTA views...")
    final_ensembled_preds = {
        'total': np.mean([d['total'] for d in all_tta_view_preds], axis=0),
        'gdm':   np.mean([d['gdm'] for d in all_tta_view_preds], axis=0),
        'green': np.mean([d['green'] for d in all_tta_view_preds], axis=0)
    }
    
    print("✓ Dự đoán hoàn tất.")
    
    del models_list, test_loader, test_dataset
    gc.collect()
    torch.cuda.empty_cache()
    
    return final_ensembled_preds, test_df_long, test_df_unique
# ===============================================================
# 6. ✍️ HÀM TẠO FILE SUBMISSION
# ===============================================================
def create_submission(preds_np, test_df_long, test_df_unique):
    """
    Hàm này nhận 3 dự đoán đã ensemble,
    tính toán 2 dự đoán còn lại,
    và định dạng file nộp bài.
    """
    if preds_np is None:
        print("Bỏ qua tạo submission do lỗi ở trên.")
        return

    print("\nĐang hậu xử lý và tạo file submission...")

    # 1. Lấy 3 dự đoán đã ensemble
    pred_total_final = preds_np['total']
    pred_gdm_final = preds_np['gdm']
    pred_green_final = preds_np['green']

    # 2. Tính 2 mục tiêu còn lại (Hậu xử lý)
    # Dùng np.maximum(0, ...) để đảm bảo không có giá trị âm
    pred_clover_final = np.maximum(0, pred_gdm_final - pred_green_final)
    pred_dead_final = np.maximum(0, pred_total_final - pred_gdm_final)

    # 3. Tạo một DataFrame "wide" chứa 5 dự đoán
    # (Đảm bảo thứ tự 5 cột giống CFG.ALL_TARGET_COLS)
    preds_wide_df = pd.DataFrame({
        'image_path': test_df_unique['image_path'],
        'Dry_Green_g': pred_green_final,
        'Dry_Dead_g': pred_dead_final,
        'Dry_Clover_g': pred_clover_final,
        'GDM_g': pred_gdm_final,
        'Dry_Total_g': pred_total_final
    })

    # 4. "Un-pivot" DataFrame (Chuyển sang dạng "long")
    # Biến nó từ 5 cột về dạng "long" (giống sample_submission)
    preds_long_df = preds_wide_df.melt(
        id_vars=['image_path'],
        value_vars=CFG.ALL_TARGET_COLS, # 5 cột mục tiêu
        var_name='target_name',        # Cột tên mục tiêu
        value_name='target'            # Cột giá trị dự đoán
    )

    # 5. Merge với file test.csv gốc (test_df_long)
    # Đây là bước quan trọng để lấy đúng 'sample_id'
    # (ví dụ: 'ID1001187975__Dry_Clover_g')
    submission_df = pd.merge(
        test_df_long[['sample_id', 'image_path', 'target_name']],
        preds_long_df,
        on=['image_path', 'target_name'],
        how='left'
    )

    # 6. Dọn dẹp và Lưu
    # Chỉ giữ lại 2 cột được yêu cầu
    submission_df = submission_df[['sample_id', 'target']]
    
    # Lưu file
    submission_df.to_csv(CFG.SUBMISSION_FILE, index=False)

    print(f"\n🎉 HOÀN TẤT! Đã lưu file submission tại: {CFG.SUBMISSION_FILE}")
    print("--- 5 hàng đầu của file submission ---")
    print(submission_df.head())
    print("\n--- 5 hàng cuối của file submission ---")
    print(submission_df.tail())
    
# ===============================================================
# 8. 🏁 CHẠY CHƯƠNG TRÌNH (Đã sửa)
# ===============================================================
if __name__ == "__main__":
    # 1. Chạy dự đoán (đã bao gồm TTA)
    all_preds_np, df_long, df_unique = run_inference_with_tta()
    
    # 2. Tạo file submission (Hàm create_submission giữ nguyên)
    create_submission(all_preds_np, df_long, df_unique)

## Model 2: Convnext tiny Without FP16 and Gradient accumulation

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import cv2
from tqdm import tqdm
import gc

# ===============================================================
# 1. ⚙️ CONFIGURATION (PHẢI GIỐNG HỆT FILE TRAINING)
# ===============================================================
class CFG:
    # --- Đường dẫn (Paths) ---
    # (Hãy điều chỉnh các đường dẫn này cho đúng với môi trường của bạn)
    BASE_PATH = '/kaggle/input/csiro-biomass'
    TEST_CSV = os.path.join(BASE_PATH, 'test.csv')
    TEST_IMAGE_DIR = os.path.join(BASE_PATH, 'test')
    
    # Thư mục chứa 5 file .pth
    MODEL_DIR = '/kaggle/input/csiro/' # Giả sử 5 file .pth nằm cùng thư mục
    SUBMISSION_FILE = 'submission2.csv'
    
    # --- Cài đặt Mô hình (PHẢI TRÙNG KHỚP) ---
    MODEL_NAME = 'convnext_tiny' # PHẢI GIỐNG HỆT LÚC TRAIN
    IMG_SIZE = 768               # PHẢI GIỐNG HỆT LÚC TRAIN
    
    # --- Cài đặt Inference ---
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 1 # Có thể tăng batch size khi inference
    NUM_WORKERS = 1
    N_FOLDS = 5
    
    # --- Mục tiêu & Loss (PHẢI TRÙNG KHỚP) ---
    # 3 mục tiêu model đã dự đoán
    TARGET_COLS = ['Dry_Total_g', 'GDM_g', 'Dry_Green_g']
    
    # 5 mục tiêu để nộp bài
    ALL_TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']

print(f"Sử dụng thiết bị: {CFG.DEVICE}")
print(f"Backbone mô hình: {CFG.MODEL_NAME}")
print(f"Kích thước ảnh inference: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}")


# ===============================================================
# 2. 🏞️ AUGMENTATIONS (CHỈ DÙNG VALIDATION)
# ===============================================================
from albumentations import (
    Compose, 
    Resize, 
    Normalize,
    HorizontalFlip, 
    VerticalFlip
)

def get_tta_transforms():
    """
    Trả về một LIST các pipeline transform cho TTA.
    Mỗi pipeline là một "view" khác nhau của ảnh.
    """
    
    # Đây là các bước chuẩn hóa cơ bản
    base_transforms = [
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        ToTensorV2()
    ]
    
    # -----------------
    # View 1: Ảnh gốc (Chỉ Resize + Normalize)
    # -----------------
    original_view = Compose([
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    # -----------------
    # View 2: Lật ngang (HFlip)
    # -----------------
    hflip_view = Compose([
        HorizontalFlip(p=1.0), # Luôn luôn lật
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    # -----------------
    # View 3: Lật dọc (VFlip)
    # -----------------
    vflip_view = Compose([
        VerticalFlip(p=1.0), # Luôn luôn lật
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    return [original_view, hflip_view, vflip_view]

print("Đã định nghĩa hàm get_tta_transforms().")


class TestBiomassDataset(Dataset):
    """
    Dataset tùy chỉnh cho ảnh test (Chiến lược "Hai luồng").
    Sửa đổi để chấp nhận một pipeline transform cụ thể cho TTA.
    """
    def __init__(self, df, transform_pipeline, image_dir):
        self.df = df
        # (SỬA ĐỔI) Chấp nhận một pipeline đã được khởi tạo
        self.transforms = transform_pipeline 
        self.image_dir = image_dir
        self.image_paths = df['image_path'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. Lấy thông tin
        img_path_suffix = self.image_paths[idx]
        
        # 2. Đọc ảnh gốc (2000x1000)
        filename = os.path.basename(img_path_suffix)
        full_path = os.path.join(self.image_dir, filename)
        
        image = cv2.imread(full_path)
        if image is None:
            print(f"Warning: Không thể đọc ảnh: {full_path}. Trả về ảnh đen.")
            image = np.zeros((1000, 2000, 3), dtype=np.uint8)
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 3. Cắt (Crop) thành 2 ảnh (Trái và Phải)
        height, width, _ = image.shape
        mid_point = width // 2
        img_left = image[:, :mid_point]
        img_right = image[:, mid_point:]
        
        # 4. Áp dụng TTA Transform (CÙNG MỘT TRANSFORM cho cả 2)
        # (Ví dụ: Cả 2 ảnh đều bị lật ngang)
        img_left_tensor = self.transforms(image=img_left)['image']
        img_right_tensor = self.transforms(image=img_right)['image']
        
        # 5. Trả về
        return img_left_tensor, img_right_tensor

# ===============================================================
# 4. 🧠 MODEL ARCHITECTURE (SAO CHÉP TỪ FILE TRAIN)
# ===============================================================
class BiomassModel(nn.Module):
    """
    Kiến trúc mô hình (Hai luồng, Ba đầu ra)
    PHẢI GIỐNG HỆT file training.
    """
    def __init__(self, model_name, pretrained, n_targets=3):
        super(BiomassModel, self).__init__()
        
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained, # Sẽ là False khi inference
            num_classes=0,
            global_pool='avg'
        )
        
        self.n_features = self.backbone.num_features
        self.n_combined_features = self.n_features * 2
        
        # --- Đầu cho Dry_Total_g ---
        self.head_total = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )
        
        # --- Đầu cho GDM_g ---
        self.head_gdm = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )
        
        # --- Đầu cho Dry_Green_g ---
        self.head_green = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )

    def forward(self, img_left, img_right):
        features_left = self.backbone(img_left)
        features_right = self.backbone(img_right)
        combined = torch.cat([features_left, features_right], dim=1)
        
        out_total = self.head_total(combined)
        out_gdm = self.head_gdm(combined)
        out_green = self.head_green(combined)
        
        return out_total, out_gdm, out_green


def predict_one_view(models_list, test_loader, device):
    """
    Hàm con: Chạy dự đoán ensemble 5-fold cho MỘT view TTA.
    """
    view_preds_3 = {'total': [], 'gdm': [], 'green': []}
    
    with torch.no_grad():
        for (img_left, img_right) in tqdm(test_loader, desc="  Predicting View", leave=False):
            img_left = img_left.to(device)
            img_right = img_right.to(device)
            
            batch_preds_3_folds = {'total': [], 'gdm': [], 'green': []}
            
            # 1. Vòng lặp Ensemble 5-Fold
            for model in models_list:
                pred_total, pred_gdm, pred_green = model(img_left, img_right)
                batch_preds_3_folds['total'].append(pred_total.cpu())
                batch_preds_3_folds['gdm'].append(pred_gdm.cpu())
                batch_preds_3_folds['green'].append(pred_green.cpu())
            
            # 2. Lấy trung bình 5 Fold
            avg_pred_total = torch.mean(torch.stack(batch_preds_3_folds['total']), dim=0)
            avg_pred_gdm = torch.mean(torch.stack(batch_preds_3_folds['gdm']), dim=0)
            avg_pred_green = torch.mean(torch.stack(batch_preds_3_folds['green']), dim=0)
            
            view_preds_3['total'].append(avg_pred_total.numpy())
            view_preds_3['gdm'].append(avg_pred_gdm.numpy())
            view_preds_3['green'].append(avg_pred_green.numpy())

    # 3. Ghép kết quả các batch của view này
    preds_np = {
        'total': np.concatenate(view_preds_3['total']).flatten(),
        'gdm':   np.concatenate(view_preds_3['gdm']).flatten(),
        'green': np.concatenate(view_preds_3['green']).flatten()
    }
    return preds_np


def run_inference_with_tta():
    """
    Hàm inference chính, thực hiện TTA x Ensemble.
    """
    print(f"\n{'='*50}")
    print(f"🚀 BẮT ĐẦU INFERENCE (với TTA) 🚀")
    print(f"{'='*50}")

    # --- 1. Tải Dữ liệu Test ---
    print(f"Đang tải {CFG.TEST_CSV}...")
    try:
        test_df_long = pd.read_csv(CFG.TEST_CSV)
        test_df_unique = test_df_long.drop_duplicates(subset=['image_path']).reset_index(drop=True)
        print(f"Tìm thấy {len(test_df_unique)} ảnh test duy nhất.")
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy {CFG.TEST_CSV}")
        return None, None, None

    # --- 2. Tải 5 Mô hình (Ensemble) ---
    print("\nĐang tải 5 mô hình đã huấn luyện...")
    models_list = []
    # (Code tải 5 mô hình... giống hệt bước 16 của file trước)
    for fold in range(CFG.N_FOLDS):
        model_path = os.path.join(CFG.MODEL_DIR, f'best_model_fold{fold}.pth')
        if not os.path.exists(model_path):
            print(f"LỖI: Không tìm thấy file mô hình: {model_path}")
            return None, None, None
        model = BiomassModel(CFG.MODEL_NAME, pretrained=False)
        try:
            model.load_state_dict(torch.load(model_path, map_location=CFG.DEVICE))
        except RuntimeError:
            state_dict = torch.load(model_path, map_location=CFG.DEVICE)
            from collections import OrderedDict
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k.replace('module.', '')
                new_state_dict[name] = v
            model.load_state_dict(new_state_dict)
        model.eval()
        model.to(CFG.DEVICE)
        models_list.append(model)
    print(f"✓ Đã tải thành công {len(models_list)} mô hình.")

    # --- 3. Vòng lặp TTA (Vòng lặp ngoài) ---
    tta_transforms = get_tta_transforms()
    print(f"\nBắt đầu dự đoán với {len(tta_transforms)} TTA views...")
    
    all_tta_view_preds = [] # List để lưu kết quả của mỗi view TTA

    for i, tta_transform in enumerate(tta_transforms):
        print(f"--- Đang chạy TTA View {i+1}/{len(tta_transforms)} ---")
        
        # Tạo Dataset/Loader MỚI cho view TTA này
        test_dataset = TestBiomassDataset(
            df=test_df_unique,
            transform_pipeline=tta_transform, # Truyền pipeline TTA
            image_dir=CFG.TEST_IMAGE_DIR
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=CFG.BATCH_SIZE,
            shuffle=False,
            num_workers=CFG.NUM_WORKERS,
            pin_memory=True
        )
        
        # Chạy ensemble 5-fold cho view này
        view_preds_np = predict_one_view(models_list, test_loader, CFG.DEVICE)
        all_tta_view_preds.append(view_preds_np)
        print(f"✓ Hoàn thành TTA View {i+1}")

    # --- 4. Ensemble (Lấy trung bình) kết quả TTA ---
    print("\nĐang ensemble kết quả của các TTA views...")
    final_ensembled_preds = {
        'total': np.mean([d['total'] for d in all_tta_view_preds], axis=0),
        'gdm':   np.mean([d['gdm'] for d in all_tta_view_preds], axis=0),
        'green': np.mean([d['green'] for d in all_tta_view_preds], axis=0)
    }
    
    print("✓ Dự đoán hoàn tất.")
    
    del models_list, test_loader, test_dataset
    gc.collect()
    torch.cuda.empty_cache()
    
    return final_ensembled_preds, test_df_long, test_df_unique
# ===============================================================
# 6. ✍️ HÀM TẠO FILE SUBMISSION
# ===============================================================
def create_submission(preds_np, test_df_long, test_df_unique):
    """
    Hàm này nhận 3 dự đoán đã ensemble,
    tính toán 2 dự đoán còn lại,
    và định dạng file nộp bài.
    """
    if preds_np is None:
        print("Bỏ qua tạo submission do lỗi ở trên.")
        return

    print("\nĐang hậu xử lý và tạo file submission...")

    # 1. Lấy 3 dự đoán đã ensemble
    pred_total_final = preds_np['total']
    pred_gdm_final = preds_np['gdm']
    pred_green_final = preds_np['green']

    # 2. Tính 2 mục tiêu còn lại (Hậu xử lý)
    # Dùng np.maximum(0, ...) để đảm bảo không có giá trị âm
    pred_clover_final = np.maximum(0, pred_gdm_final - pred_green_final)
    pred_dead_final = np.maximum(0, pred_total_final - pred_gdm_final)

    # 3. Tạo một DataFrame "wide" chứa 5 dự đoán
    # (Đảm bảo thứ tự 5 cột giống CFG.ALL_TARGET_COLS)
    preds_wide_df = pd.DataFrame({
        'image_path': test_df_unique['image_path'],
        'Dry_Green_g': pred_green_final,
        'Dry_Dead_g': pred_dead_final,
        'Dry_Clover_g': pred_clover_final,
        'GDM_g': pred_gdm_final,
        'Dry_Total_g': pred_total_final
    })

    # 4. "Un-pivot" DataFrame (Chuyển sang dạng "long")
    # Biến nó từ 5 cột về dạng "long" (giống sample_submission)
    preds_long_df = preds_wide_df.melt(
        id_vars=['image_path'],
        value_vars=CFG.ALL_TARGET_COLS, # 5 cột mục tiêu
        var_name='target_name',        # Cột tên mục tiêu
        value_name='target'            # Cột giá trị dự đoán
    )

    # 5. Merge với file test.csv gốc (test_df_long)
    # Đây là bước quan trọng để lấy đúng 'sample_id'
    # (ví dụ: 'ID1001187975__Dry_Clover_g')
    submission_df = pd.merge(
        test_df_long[['sample_id', 'image_path', 'target_name']],
        preds_long_df,
        on=['image_path', 'target_name'],
        how='left'
    )

    # 6. Dọn dẹp và Lưu
    # Chỉ giữ lại 2 cột được yêu cầu
    submission_df = submission_df[['sample_id', 'target']]
    
    # Lưu file
    submission_df.to_csv(CFG.SUBMISSION_FILE, index=False)

    print(f"\n🎉 HOÀN TẤT! Đã lưu file submission tại: {CFG.SUBMISSION_FILE}")
    print("--- 5 hàng đầu của file submission ---")
    print(submission_df.head())
    print("\n--- 5 hàng cuối của file submission ---")
    print(submission_df.tail())
    
# ===============================================================
# 8. 🏁 CHẠY CHƯƠNG TRÌNH (Đã sửa)
# ===============================================================
if __name__ == "__main__":
    # 1. Chạy dự đoán (đã bao gồm TTA)
    all_preds_np, df_long, df_unique = run_inference_with_tta()
    
    # 2. Tạo file submission (Hàm create_submission giữ nguyên)
    create_submission(all_preds_np, df_long, df_unique)

## COATNET

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import cv2
from tqdm import tqdm
import gc

# ===============================================================
# 1. ⚙️ CONFIGURATION (PHẢI GIỐNG HỆT FILE TRAINING)
# ===============================================================
class CFG:
    # --- Đường dẫn (Paths) ---
    # (Hãy điều chỉnh các đường dẫn này cho đúng với môi trường của bạn)
    BASE_PATH = '/kaggle/input/csiro-biomass'
    TEST_CSV = os.path.join(BASE_PATH, 'test.csv')
    TEST_IMAGE_DIR = os.path.join(BASE_PATH, 'test')
    
    # Thư mục chứa 5 file .pth
    MODEL_DIR = '/kaggle/input/csiro-coatnet-with-oof-train/' # Giả sử 5 file .pth nằm cùng thư mục
    SUBMISSION_FILE = 'submission3.csv'
    
    # --- Cài đặt Mô hình (PHẢI TRÙNG KHỚP) ---
    MODEL_NAME = 'coatnet_rmlp_2_rw_384.sw_in12k_ft_in1k' # PHẢI GIỐNG HỆT LÚC TRAIN
    IMG_SIZE = 384               # PHẢI GIỐNG HỆT LÚC TRAIN
    
    # --- Cài đặt Inference ---
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 1 # Có thể tăng batch size khi inference
    NUM_WORKERS = 1
    N_FOLDS = 5
    
    # --- Mục tiêu & Loss (PHẢI TRÙNG KHỚP) ---
    # 3 mục tiêu model đã dự đoán
    TARGET_COLS = ['Dry_Total_g', 'GDM_g', 'Dry_Green_g']
    
    # 5 mục tiêu để nộp bài
    ALL_TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']

print(f"Sử dụng thiết bị: {CFG.DEVICE}")
print(f"Backbone mô hình: {CFG.MODEL_NAME}")
print(f"Kích thước ảnh inference: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}")


# ===============================================================
# 2. 🏞️ AUGMENTATIONS (CHỈ DÙNG VALIDATION)
# ===============================================================
from albumentations import (
    Compose, 
    Resize, 
    Normalize,
    HorizontalFlip, 
    VerticalFlip
)

def get_tta_transforms():
    """
    Trả về một LIST các pipeline transform cho TTA.
    Mỗi pipeline là một "view" khác nhau của ảnh.
    """
    
    # Đây là các bước chuẩn hóa cơ bản
    base_transforms = [
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        ToTensorV2()
    ]
    
    # -----------------
    # View 1: Ảnh gốc (Chỉ Resize + Normalize)
    # -----------------
    original_view = Compose([
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    # -----------------
    # View 2: Lật ngang (HFlip)
    # -----------------
    hflip_view = Compose([
        HorizontalFlip(p=1.0), # Luôn luôn lật
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    # -----------------
    # View 3: Lật dọc (VFlip)
    # -----------------
    vflip_view = Compose([
        VerticalFlip(p=1.0), # Luôn luôn lật
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    return [original_view, hflip_view, vflip_view]

print("Đã định nghĩa hàm get_tta_transforms().")


class TestBiomassDataset(Dataset):
    """
    Dataset tùy chỉnh cho ảnh test (Chiến lược "Hai luồng").
    Sửa đổi để chấp nhận một pipeline transform cụ thể cho TTA.
    """
    def __init__(self, df, transform_pipeline, image_dir):
        self.df = df
        # (SỬA ĐỔI) Chấp nhận một pipeline đã được khởi tạo
        self.transforms = transform_pipeline 
        self.image_dir = image_dir
        self.image_paths = df['image_path'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. Lấy thông tin
        img_path_suffix = self.image_paths[idx]
        
        # 2. Đọc ảnh gốc (2000x1000)
        filename = os.path.basename(img_path_suffix)
        full_path = os.path.join(self.image_dir, filename)
        
        image = cv2.imread(full_path)
        if image is None:
            print(f"Warning: Không thể đọc ảnh: {full_path}. Trả về ảnh đen.")
            image = np.zeros((1000, 2000, 3), dtype=np.uint8)
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 3. Cắt (Crop) thành 2 ảnh (Trái và Phải)
        height, width, _ = image.shape
        mid_point = width // 2
        img_left = image[:, :mid_point]
        img_right = image[:, mid_point:]
        
        # 4. Áp dụng TTA Transform (CÙNG MỘT TRANSFORM cho cả 2)
        # (Ví dụ: Cả 2 ảnh đều bị lật ngang)
        img_left_tensor = self.transforms(image=img_left)['image']
        img_right_tensor = self.transforms(image=img_right)['image']
        
        # 5. Trả về
        return img_left_tensor, img_right_tensor

# ===============================================================
# 4. 🧠 MODEL ARCHITECTURE (SAO CHÉP TỪ FILE TRAIN)
# ===============================================================
class BiomassModel(nn.Module):
    """
    Kiến trúc mô hình (Hai luồng, Ba đầu ra)
    PHẢI GIỐNG HỆT file training.
    """
    def __init__(self, model_name, pretrained, n_targets=3):
        super(BiomassModel, self).__init__()
        
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained, # Sẽ là False khi inference
            num_classes=0,
            global_pool='avg'
        )
        
        self.n_features = self.backbone.num_features
        self.n_combined_features = self.n_features * 2
        
        # --- Đầu cho Dry_Total_g ---
        self.head_total = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )
        
        # --- Đầu cho GDM_g ---
        self.head_gdm = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )
        
        # --- Đầu cho Dry_Green_g ---
        self.head_green = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )

    def forward(self, img_left, img_right):
        features_left = self.backbone(img_left)
        features_right = self.backbone(img_right)
        combined = torch.cat([features_left, features_right], dim=1)
        
        out_total = self.head_total(combined)
        out_gdm = self.head_gdm(combined)
        out_green = self.head_green(combined)
        
        return out_total, out_gdm, out_green


def predict_one_view(models_list, test_loader, device):
    """
    Hàm con: Chạy dự đoán ensemble 5-fold cho MỘT view TTA.
    """
    view_preds_3 = {'total': [], 'gdm': [], 'green': []}
    
    with torch.no_grad():
        for (img_left, img_right) in tqdm(test_loader, desc="  Predicting View", leave=False):
            img_left = img_left.to(device)
            img_right = img_right.to(device)
            
            batch_preds_3_folds = {'total': [], 'gdm': [], 'green': []}
            
            # 1. Vòng lặp Ensemble 5-Fold
            for model in models_list:
                pred_total, pred_gdm, pred_green = model(img_left, img_right)
                batch_preds_3_folds['total'].append(pred_total.cpu())
                batch_preds_3_folds['gdm'].append(pred_gdm.cpu())
                batch_preds_3_folds['green'].append(pred_green.cpu())
            
            # 2. Lấy trung bình 5 Fold
            avg_pred_total = torch.mean(torch.stack(batch_preds_3_folds['total']), dim=0)
            avg_pred_gdm = torch.mean(torch.stack(batch_preds_3_folds['gdm']), dim=0)
            avg_pred_green = torch.mean(torch.stack(batch_preds_3_folds['green']), dim=0)
            
            view_preds_3['total'].append(avg_pred_total.numpy())
            view_preds_3['gdm'].append(avg_pred_gdm.numpy())
            view_preds_3['green'].append(avg_pred_green.numpy())

    # 3. Ghép kết quả các batch của view này
    preds_np = {
        'total': np.concatenate(view_preds_3['total']).flatten(),
        'gdm':   np.concatenate(view_preds_3['gdm']).flatten(),
        'green': np.concatenate(view_preds_3['green']).flatten()
    }
    return preds_np


def run_inference_with_tta():
    """
    Hàm inference chính, thực hiện TTA x Ensemble.
    """
    print(f"\n{'='*50}")
    print(f"🚀 BẮT ĐẦU INFERENCE (với TTA) 🚀")
    print(f"{'='*50}")

    # --- 1. Tải Dữ liệu Test ---
    print(f"Đang tải {CFG.TEST_CSV}...")
    try:
        test_df_long = pd.read_csv(CFG.TEST_CSV)
        test_df_unique = test_df_long.drop_duplicates(subset=['image_path']).reset_index(drop=True)
        print(f"Tìm thấy {len(test_df_unique)} ảnh test duy nhất.")
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy {CFG.TEST_CSV}")
        return None, None, None

    # --- 2. Tải 5 Mô hình (Ensemble) ---
    print("\nĐang tải 5 mô hình đã huấn luyện...")
    models_list = []
    # (Code tải 5 mô hình... giống hệt bước 16 của file trước)
    for fold in range(CFG.N_FOLDS):
        model_path = os.path.join(CFG.MODEL_DIR, f'best_model_fold{fold}.pth')
        if not os.path.exists(model_path):
            print(f"LỖI: Không tìm thấy file mô hình: {model_path}")
            return None, None, None
        model = BiomassModel(CFG.MODEL_NAME, pretrained=False)
        try:
            model.load_state_dict(torch.load(model_path, map_location=CFG.DEVICE))
        except RuntimeError:
            state_dict = torch.load(model_path, map_location=CFG.DEVICE)
            from collections import OrderedDict
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k.replace('module.', '')
                new_state_dict[name] = v
            model.load_state_dict(new_state_dict)
        model.eval()
        model.to(CFG.DEVICE)
        models_list.append(model)
    print(f"✓ Đã tải thành công {len(models_list)} mô hình.")

    # --- 3. Vòng lặp TTA (Vòng lặp ngoài) ---
    tta_transforms = get_tta_transforms()
    print(f"\nBắt đầu dự đoán với {len(tta_transforms)} TTA views...")
    
    all_tta_view_preds = [] # List để lưu kết quả của mỗi view TTA

    for i, tta_transform in enumerate(tta_transforms):
        print(f"--- Đang chạy TTA View {i+1}/{len(tta_transforms)} ---")
        
        # Tạo Dataset/Loader MỚI cho view TTA này
        test_dataset = TestBiomassDataset(
            df=test_df_unique,
            transform_pipeline=tta_transform, # Truyền pipeline TTA
            image_dir=CFG.TEST_IMAGE_DIR
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=CFG.BATCH_SIZE,
            shuffle=False,
            num_workers=CFG.NUM_WORKERS,
            pin_memory=True
        )
        
        # Chạy ensemble 5-fold cho view này
        view_preds_np = predict_one_view(models_list, test_loader, CFG.DEVICE)
        all_tta_view_preds.append(view_preds_np)
        print(f"✓ Hoàn thành TTA View {i+1}")

    # --- 4. Ensemble (Lấy trung bình) kết quả TTA ---
    print("\nĐang ensemble kết quả của các TTA views...")
    final_ensembled_preds = {
        'total': np.mean([d['total'] for d in all_tta_view_preds], axis=0),
        'gdm':   np.mean([d['gdm'] for d in all_tta_view_preds], axis=0),
        'green': np.mean([d['green'] for d in all_tta_view_preds], axis=0)
    }
    
    print("✓ Dự đoán hoàn tất.")
    
    del models_list, test_loader, test_dataset
    gc.collect()
    torch.cuda.empty_cache()
    
    return final_ensembled_preds, test_df_long, test_df_unique
# ===============================================================
# 6. ✍️ HÀM TẠO FILE SUBMISSION
# ===============================================================
def create_submission(preds_np, test_df_long, test_df_unique):
    """
    Hàm này nhận 3 dự đoán đã ensemble,
    tính toán 2 dự đoán còn lại,
    và định dạng file nộp bài.
    """
    if preds_np is None:
        print("Bỏ qua tạo submission do lỗi ở trên.")
        return

    print("\nĐang hậu xử lý và tạo file submission...")

    # 1. Lấy 3 dự đoán đã ensemble
    pred_total_final = preds_np['total']
    pred_gdm_final = preds_np['gdm']
    pred_green_final = preds_np['green']

    # 2. Tính 2 mục tiêu còn lại (Hậu xử lý)
    # Dùng np.maximum(0, ...) để đảm bảo không có giá trị âm
    pred_clover_final = np.maximum(0, pred_gdm_final - pred_green_final)
    pred_dead_final = np.maximum(0, pred_total_final - pred_gdm_final)

    # 3. Tạo một DataFrame "wide" chứa 5 dự đoán
    # (Đảm bảo thứ tự 5 cột giống CFG.ALL_TARGET_COLS)
    preds_wide_df = pd.DataFrame({
        'image_path': test_df_unique['image_path'],
        'Dry_Green_g': pred_green_final,
        'Dry_Dead_g': pred_dead_final,
        'Dry_Clover_g': pred_clover_final,
        'GDM_g': pred_gdm_final,
        'Dry_Total_g': pred_total_final
    })

    # 4. "Un-pivot" DataFrame (Chuyển sang dạng "long")
    # Biến nó từ 5 cột về dạng "long" (giống sample_submission)
    preds_long_df = preds_wide_df.melt(
        id_vars=['image_path'],
        value_vars=CFG.ALL_TARGET_COLS, # 5 cột mục tiêu
        var_name='target_name',        # Cột tên mục tiêu
        value_name='target'            # Cột giá trị dự đoán
    )

    # 5. Merge với file test.csv gốc (test_df_long)
    # Đây là bước quan trọng để lấy đúng 'sample_id'
    # (ví dụ: 'ID1001187975__Dry_Clover_g')
    submission_df = pd.merge(
        test_df_long[['sample_id', 'image_path', 'target_name']],
        preds_long_df,
        on=['image_path', 'target_name'],
        how='left'
    )

    # 6. Dọn dẹp và Lưu
    # Chỉ giữ lại 2 cột được yêu cầu
    submission_df = submission_df[['sample_id', 'target']]
    
    # Lưu file
    submission_df.to_csv(CFG.SUBMISSION_FILE, index=False)

    print(f"\n🎉 HOÀN TẤT! Đã lưu file submission tại: {CFG.SUBMISSION_FILE}")
    print("--- 5 hàng đầu của file submission ---")
    print(submission_df.head())
    print("\n--- 5 hàng cuối của file submission ---")
    print(submission_df.tail())
    
# ===============================================================
# 8. 🏁 CHẠY CHƯƠNG TRÌNH (Đã sửa)
# ===============================================================
if __name__ == "__main__":
    # 1. Chạy dự đoán (đã bao gồm TTA)
    all_preds_np, df_long, df_unique = run_inference_with_tta()
    
    # 2. Tạo file submission (Hàm create_submission giữ nguyên)
    create_submission(all_preds_np, df_long, df_unique)

## Convnext Tiny 512 Image_Size

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import cv2
from tqdm import tqdm
import gc

# ===============================================================
# 1. ⚙️ CONFIGURATION (PHẢI GIỐNG HỆT FILE TRAINING)
# ===============================================================
class CFG:
    # --- Đường dẫn (Paths) ---
    # (Hãy điều chỉnh các đường dẫn này cho đúng với môi trường của bạn)
    BASE_PATH = '/kaggle/input/csiro-biomass'
    TEST_CSV = os.path.join(BASE_PATH, 'test.csv')
    TEST_IMAGE_DIR = os.path.join(BASE_PATH, 'test')
    
    # Thư mục chứa 5 file .pth
    MODEL_DIR = '/kaggle/input/csiro-train-convnext-tiny-512-imsize/' # Giả sử 5 file .pth nằm cùng thư mục
    SUBMISSION_FILE = 'submission_convnext_512_Imsize.csv'
    
    # --- Cài đặt Mô hình (PHẢI TRÙNG KHỚP) ---
    MODEL_NAME = 'convnext_tiny' # PHẢI GIỐNG HỆT LÚC TRAIN
    IMG_SIZE = 512               # PHẢI GIỐNG HỆT LÚC TRAIN
    
    # --- Cài đặt Inference ---
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 1 # Có thể tăng batch size khi inference
    NUM_WORKERS = 1
    N_FOLDS = 5
    
    # --- Mục tiêu & Loss (PHẢI TRÙNG KHỚP) ---
    # 3 mục tiêu model đã dự đoán
    TARGET_COLS = ['Dry_Total_g', 'GDM_g', 'Dry_Green_g']
    
    # 5 mục tiêu để nộp bài
    ALL_TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']

print(f"Sử dụng thiết bị: {CFG.DEVICE}")
print(f"Backbone mô hình: {CFG.MODEL_NAME}")
print(f"Kích thước ảnh inference: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}")


# ===============================================================
# 2. 🏞️ AUGMENTATIONS (CHỈ DÙNG VALIDATION)
# ===============================================================
from albumentations import (
    Compose, 
    Resize, 
    Normalize,
    HorizontalFlip, 
    VerticalFlip
)

def get_tta_transforms():
    """
    Trả về một LIST các pipeline transform cho TTA.
    Mỗi pipeline là một "view" khác nhau của ảnh.
    """
    
    # Đây là các bước chuẩn hóa cơ bản
    base_transforms = [
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        ToTensorV2()
    ]
    
    # -----------------
    # View 1: Ảnh gốc (Chỉ Resize + Normalize)
    # -----------------
    original_view = Compose([
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    # -----------------
    # View 2: Lật ngang (HFlip)
    # -----------------
    hflip_view = Compose([
        HorizontalFlip(p=1.0), # Luôn luôn lật
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    # -----------------
    # View 3: Lật dọc (VFlip)
    # -----------------
    vflip_view = Compose([
        VerticalFlip(p=1.0), # Luôn luôn lật
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    return [original_view, hflip_view, vflip_view]

print("Đã định nghĩa hàm get_tta_transforms().")


class TestBiomassDataset(Dataset):
    """
    Dataset tùy chỉnh cho ảnh test (Chiến lược "Hai luồng").
    Sửa đổi để chấp nhận một pipeline transform cụ thể cho TTA.
    """
    def __init__(self, df, transform_pipeline, image_dir):
        self.df = df
        # (SỬA ĐỔI) Chấp nhận một pipeline đã được khởi tạo
        self.transforms = transform_pipeline 
        self.image_dir = image_dir
        self.image_paths = df['image_path'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. Lấy thông tin
        img_path_suffix = self.image_paths[idx]
        
        # 2. Đọc ảnh gốc (2000x1000)
        filename = os.path.basename(img_path_suffix)
        full_path = os.path.join(self.image_dir, filename)
        
        image = cv2.imread(full_path)
        if image is None:
            print(f"Warning: Không thể đọc ảnh: {full_path}. Trả về ảnh đen.")
            image = np.zeros((1000, 2000, 3), dtype=np.uint8)
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 3. Cắt (Crop) thành 2 ảnh (Trái và Phải)
        height, width, _ = image.shape
        mid_point = width // 2
        img_left = image[:, :mid_point]
        img_right = image[:, mid_point:]
        
        # 4. Áp dụng TTA Transform (CÙNG MỘT TRANSFORM cho cả 2)
        # (Ví dụ: Cả 2 ảnh đều bị lật ngang)
        img_left_tensor = self.transforms(image=img_left)['image']
        img_right_tensor = self.transforms(image=img_right)['image']
        
        # 5. Trả về
        return img_left_tensor, img_right_tensor

# ===============================================================
# 4. 🧠 MODEL ARCHITECTURE (SAO CHÉP TỪ FILE TRAIN)
# ===============================================================
class BiomassModel(nn.Module):
    """
    Kiến trúc mô hình (Hai luồng, Ba đầu ra)
    PHẢI GIỐNG HỆT file training.
    """
    def __init__(self, model_name, pretrained, n_targets=3):
        super(BiomassModel, self).__init__()
        
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained, # Sẽ là False khi inference
            num_classes=0,
            global_pool='avg'
        )
        
        self.n_features = self.backbone.num_features
        self.n_combined_features = self.n_features * 2
        
        # --- Đầu cho Dry_Total_g ---
        self.head_total = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )
        
        # --- Đầu cho GDM_g ---
        self.head_gdm = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )
        
        # --- Đầu cho Dry_Green_g ---
        self.head_green = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )

    def forward(self, img_left, img_right):
        features_left = self.backbone(img_left)
        features_right = self.backbone(img_right)
        combined = torch.cat([features_left, features_right], dim=1)
        
        out_total = self.head_total(combined)
        out_gdm = self.head_gdm(combined)
        out_green = self.head_green(combined)
        
        return out_total, out_gdm, out_green


def predict_one_view(models_list, test_loader, device):
    """
    Hàm con: Chạy dự đoán ensemble 5-fold cho MỘT view TTA.
    """
    view_preds_3 = {'total': [], 'gdm': [], 'green': []}
    
    with torch.no_grad():
        for (img_left, img_right) in tqdm(test_loader, desc="  Predicting View", leave=False):
            img_left = img_left.to(device)
            img_right = img_right.to(device)
            
            batch_preds_3_folds = {'total': [], 'gdm': [], 'green': []}
            
            # 1. Vòng lặp Ensemble 5-Fold
            for model in models_list:
                pred_total, pred_gdm, pred_green = model(img_left, img_right)
                batch_preds_3_folds['total'].append(pred_total.cpu())
                batch_preds_3_folds['gdm'].append(pred_gdm.cpu())
                batch_preds_3_folds['green'].append(pred_green.cpu())
            
            # 2. Lấy trung bình 5 Fold
            avg_pred_total = torch.mean(torch.stack(batch_preds_3_folds['total']), dim=0)
            avg_pred_gdm = torch.mean(torch.stack(batch_preds_3_folds['gdm']), dim=0)
            avg_pred_green = torch.mean(torch.stack(batch_preds_3_folds['green']), dim=0)
            
            view_preds_3['total'].append(avg_pred_total.numpy())
            view_preds_3['gdm'].append(avg_pred_gdm.numpy())
            view_preds_3['green'].append(avg_pred_green.numpy())

    # 3. Ghép kết quả các batch của view này
    preds_np = {
        'total': np.concatenate(view_preds_3['total']).flatten(),
        'gdm':   np.concatenate(view_preds_3['gdm']).flatten(),
        'green': np.concatenate(view_preds_3['green']).flatten()
    }
    return preds_np


def run_inference_with_tta():
    """
    Hàm inference chính, thực hiện TTA x Ensemble.
    """
    print(f"\n{'='*50}")
    print(f"🚀 BẮT ĐẦU INFERENCE (với TTA) 🚀")
    print(f"{'='*50}")

    # --- 1. Tải Dữ liệu Test ---
    print(f"Đang tải {CFG.TEST_CSV}...")
    try:
        test_df_long = pd.read_csv(CFG.TEST_CSV)
        test_df_unique = test_df_long.drop_duplicates(subset=['image_path']).reset_index(drop=True)
        print(f"Tìm thấy {len(test_df_unique)} ảnh test duy nhất.")
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy {CFG.TEST_CSV}")
        return None, None, None

    # --- 2. Tải 5 Mô hình (Ensemble) ---
    print("\nĐang tải 5 mô hình đã huấn luyện...")
    models_list = []
    # (Code tải 5 mô hình... giống hệt bước 16 của file trước)
    for fold in range(CFG.N_FOLDS):
        model_path = os.path.join(CFG.MODEL_DIR, f'best_model_fold{fold}.pth')
        if not os.path.exists(model_path):
            print(f"LỖI: Không tìm thấy file mô hình: {model_path}")
            return None, None, None
        model = BiomassModel(CFG.MODEL_NAME, pretrained=False)
        try:
            model.load_state_dict(torch.load(model_path, map_location=CFG.DEVICE))
        except RuntimeError:
            state_dict = torch.load(model_path, map_location=CFG.DEVICE)
            from collections import OrderedDict
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k.replace('module.', '')
                new_state_dict[name] = v
            model.load_state_dict(new_state_dict)
        model.eval()
        model.to(CFG.DEVICE)
        models_list.append(model)
    print(f"✓ Đã tải thành công {len(models_list)} mô hình.")

    # --- 3. Vòng lặp TTA (Vòng lặp ngoài) ---
    tta_transforms = get_tta_transforms()
    print(f"\nBắt đầu dự đoán với {len(tta_transforms)} TTA views...")
    
    all_tta_view_preds = [] # List để lưu kết quả của mỗi view TTA

    for i, tta_transform in enumerate(tta_transforms):
        print(f"--- Đang chạy TTA View {i+1}/{len(tta_transforms)} ---")
        
        # Tạo Dataset/Loader MỚI cho view TTA này
        test_dataset = TestBiomassDataset(
            df=test_df_unique,
            transform_pipeline=tta_transform, # Truyền pipeline TTA
            image_dir=CFG.TEST_IMAGE_DIR
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=CFG.BATCH_SIZE,
            shuffle=False,
            num_workers=CFG.NUM_WORKERS,
            pin_memory=True
        )
        
        # Chạy ensemble 5-fold cho view này
        view_preds_np = predict_one_view(models_list, test_loader, CFG.DEVICE)
        all_tta_view_preds.append(view_preds_np)
        print(f"✓ Hoàn thành TTA View {i+1}")

    # --- 4. Ensemble (Lấy trung bình) kết quả TTA ---
    print("\nĐang ensemble kết quả của các TTA views...")
    final_ensembled_preds = {
        'total': np.mean([d['total'] for d in all_tta_view_preds], axis=0),
        'gdm':   np.mean([d['gdm'] for d in all_tta_view_preds], axis=0),
        'green': np.mean([d['green'] for d in all_tta_view_preds], axis=0)
    }
    
    print("✓ Dự đoán hoàn tất.")
    
    del models_list, test_loader, test_dataset
    gc.collect()
    torch.cuda.empty_cache()
    
    return final_ensembled_preds, test_df_long, test_df_unique
# ===============================================================
# 6. ✍️ HÀM TẠO FILE SUBMISSION
# ===============================================================
def create_submission(preds_np, test_df_long, test_df_unique):
    """
    Hàm này nhận 3 dự đoán đã ensemble,
    tính toán 2 dự đoán còn lại,
    và định dạng file nộp bài.
    """
    if preds_np is None:
        print("Bỏ qua tạo submission do lỗi ở trên.")
        return

    print("\nĐang hậu xử lý và tạo file submission...")

    # 1. Lấy 3 dự đoán đã ensemble
    pred_total_final = preds_np['total']
    pred_gdm_final = preds_np['gdm']
    pred_green_final = preds_np['green']

    # 2. Tính 2 mục tiêu còn lại (Hậu xử lý)
    # Dùng np.maximum(0, ...) để đảm bảo không có giá trị âm
    pred_clover_final = np.maximum(0, pred_gdm_final - pred_green_final)
    pred_dead_final = np.maximum(0, pred_total_final - pred_gdm_final)

    # 3. Tạo một DataFrame "wide" chứa 5 dự đoán
    # (Đảm bảo thứ tự 5 cột giống CFG.ALL_TARGET_COLS)
    preds_wide_df = pd.DataFrame({
        'image_path': test_df_unique['image_path'],
        'Dry_Green_g': pred_green_final,
        'Dry_Dead_g': pred_dead_final,
        'Dry_Clover_g': pred_clover_final,
        'GDM_g': pred_gdm_final,
        'Dry_Total_g': pred_total_final
    })

    # 4. "Un-pivot" DataFrame (Chuyển sang dạng "long")
    # Biến nó từ 5 cột về dạng "long" (giống sample_submission)
    preds_long_df = preds_wide_df.melt(
        id_vars=['image_path'],
        value_vars=CFG.ALL_TARGET_COLS, # 5 cột mục tiêu
        var_name='target_name',        # Cột tên mục tiêu
        value_name='target'            # Cột giá trị dự đoán
    )

    # 5. Merge với file test.csv gốc (test_df_long)
    # Đây là bước quan trọng để lấy đúng 'sample_id'
    # (ví dụ: 'ID1001187975__Dry_Clover_g')
    submission_df = pd.merge(
        test_df_long[['sample_id', 'image_path', 'target_name']],
        preds_long_df,
        on=['image_path', 'target_name'],
        how='left'
    )

    # 6. Dọn dẹp và Lưu
    # Chỉ giữ lại 2 cột được yêu cầu
    submission_df = submission_df[['sample_id', 'target']]
    
    # Lưu file
    submission_df.to_csv(CFG.SUBMISSION_FILE, index=False)

    print(f"\n🎉 HOÀN TẤT! Đã lưu file submission tại: {CFG.SUBMISSION_FILE}")
    print("--- 5 hàng đầu của file submission ---")
    print(submission_df.head())
    print("\n--- 5 hàng cuối của file submission ---")
    print(submission_df.tail())
    
# ===============================================================
# 8. 🏁 CHẠY CHƯƠNG TRÌNH (Đã sửa)
# ===============================================================
if __name__ == "__main__":
    # 1. Chạy dự đoán (đã bao gồm TTA)
    all_preds_np, df_long, df_unique = run_inference_with_tta()
    
    # 2. Tạo file submission (Hàm create_submission giữ nguyên)
    create_submission(all_preds_np, df_long, df_unique)

## Model 4  :DINO with LgbmRegressor

In [ ]:
import pandas as pd
import numpy as np
import torch
import torchvision
import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset, Dataset
from PIL import Image

!cp -r "/kaggle/input/rsna-models/facebookresearch_dinov2_main (1)/root/.cache/torch/hub/facebookresearch_dinov2_main" /kaggle/working/dinov2
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
from transformers import AutoImageProcessor, AutoModel
# processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/base/1/')
# model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/base/1/')
processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/giant/1')
model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/giant/1')
model = model.cuda()
embeds = []
targets = [[] for i in range(5)]
counter = 0
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
#transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224)), transforms.Normalize(mean, std)])
train_df = pd.read_csv("/kaggle/input/csiro-biomass/train.csv")
root = "/kaggle/input/csiro-biomass/"
for i in range(len(train_df)):
    entry = train_df.iloc[i]
    file_path = root + entry['image_path']
    y = torch.tensor([[entry['target']]])
    targets[i % 5].append(y)
    if i % 5 == 0:
        img = Image.open(file_path)
        x = torch.tensor(processor(img).pixel_values)
        with torch.no_grad():
            x = x.cuda()
            embeds.append(model(x).pooler_output.cpu())
            counter += 1
            if counter % 100 == 0:
                print(f"{counter} batches processed.")

# --- Start of Modified Section ---
import random
import numpy as np
from sklearn.metrics import r2_score
# Import LGBMRegressor instead of Lasso
from lightgbm import LGBMRegressor 
# Create indices and shuffle once
lst = list(range(len(embeds)))
random.seed(42)
random.shuffle(lst)

# Create multiple random 80/20 splits
n_splits = 5
splits = []
for i in range(n_splits):
    # Reshuffle for each split while maintaining same splits across targets
    temp_lst = lst.copy()
    random.seed(42 + i)  # Different seed for each split
    random.shuffle(temp_lst)
    
    split_point = int(len(temp_lst) * 0.8)
    train_idxs = temp_lst[:split_point]
    val_idxs = temp_lst[split_point:]
    splits.append((train_idxs, val_idxs))

# Convert embeds to numpy array once for efficiency
embeds_np = np.array(torch.cat(embeds))
regressors = [[None for i in range(5)] for j in range(5)]

# Now iterate through each target
for i in range(5):
    print(f"\n=== Target {i+1} ===")
    targets_np = np.array(torch.cat(targets[i]))
    
    split_scores = []
    
    for split_idx, (train_idxs, val_idxs) in enumerate(splits):
        print(f"Fold {split_idx+1}:")
        X_train, y_train = embeds_np[train_idxs], targets_np[train_idxs]
        X_val, y_val = embeds_np[val_idxs], targets_np[val_idxs]
        
        # Replace Lasso() with LGBMRegressor() and set random_state for determinism
        reg = LGBMRegressor(random_state=42) 
        reg.fit(X_train, y_train.ravel()) # .ravel() is used to ensure y_train is 1D for LightGBM
        
        train_preds = reg.predict(X_train)
        train_preds[train_preds < 0.0] = 0.0
        train_r2 = r2_score(y_train, train_preds)
        
        val_preds = reg.predict(X_val)
        val_preds[val_preds < 0.0] = 0.0
        val_r2 = r2_score(y_val, val_preds)
        
        print(f"  Train R²: {train_r2:.4f}")
        print(f"  Val R²: {val_r2:.4f}")
        split_scores.append((train_r2, val_r2))
        regressors[i][split_idx] = reg
    
    # Print summary for this target
    avg_train_r2 = np.mean([score[0] for score in split_scores])
    avg_val_r2 = np.mean([score[1] for score in split_scores])
    print(f"\nTarget {i+1} Average:")
    print(f"  Avg Train R²: {avg_train_r2:.4f}")
    print(f"  Avg Val R²: {avg_val_r2:.4f}")
# --- End of Modified Section ---

mapping = {"Dry_Clover_g": 0, "Dry_Dead_g": 1, "Dry_Green_g": 2, "Dry_Total_g": 3, "GDM_g": 4}
test_embeds = {}
counter = 0
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224)), transforms.Normalize(mean, std)])
test_df = pd.read_csv("/kaggle/input/csiro-biomass/test.csv")
root = "/kaggle/input/csiro-biomass/"
sample_ids = []
for i in range(len(test_df)):
    entry = test_df.iloc[i]
    file_path = root + entry['image_path']
    sample_id = entry['sample_id']
    #y = torch.tensor([[entry['target']]])
    if sample_id not in sample_ids:
        img = Image.open(file_path)
        x = torch.tensor(processor(img).pixel_values)
        with torch.no_grad():
            x = x.cuda()
            test_embeds[sample_id.split("_")[0]] = model(x).pooler_output.cpu()
            counter += 1
        sample_ids.append(sample_id)
    if counter % 100 == 0:
        print(f"{counter} batches processed.")

predictions = []
sample_ids = []
test_df = pd.read_csv("/kaggle/input/csiro-biomass/test.csv")
for i in range(len(test_df)):
    try:
        entry = test_df.iloc[i]
        X = np.array(test_embeds[entry['sample_id'].split("__")[0]])
        sample_ids.append(entry['sample_id'])
        models = regressors[mapping[entry['sample_id'].split("__")[1]]]
        prediction = 0.0
        for item in models:
            single_pred = item.predict(X)
            if single_pred < 0.0:
                single_pred = 0.0
            prediction += single_pred
        prediction = prediction / 5
        predictions.append(float(prediction))
    except Exception as e:
        predictions.append(0.0)
        
%cd /kaggle/working
submission = pd.DataFrame({
    'sample_id': sample_ids,
    'target': predictions})
submission.to_csv('submission4.csv', index=False)

## Dino Base with Lgbm regressor

In [ ]:
import pandas as pd
import numpy as np
import torch
import torchvision
import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset, Dataset
from PIL import Image
!cp -r "/kaggle/input/rsna-models/facebookresearch_dinov2_main (1)/root/.cache/torch/hub/facebookresearch_dinov2_main" /kaggle/working/dinov2
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]


from transformers import AutoImageProcessor, AutoModel
processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/base/1/')
model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/base/1/')
model = model.cuda()


embeds = []
targets = [[] for i in range(5)]
counter = 0
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
#transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224)), transforms.Normalize(mean, std)])
train_df = pd.read_csv("/kaggle/input/csiro-biomass/train.csv")
root = "/kaggle/input/csiro-biomass/"
for i in range(len(train_df)):
    entry = train_df.iloc[i]
    file_path = root + entry['image_path']
    y = torch.tensor([[entry['target']]])
    targets[i % 5].append(y)
    if i % 5 == 0:
        img = Image.open(file_path)
        x = torch.tensor(processor(img).pixel_values)
        with torch.no_grad():
            x = x.cuda()
            embeds.append(model(x).pooler_output.cpu())
            counter += 1
            if counter % 100 == 0:
                print(f"{counter} batches processed.")


import random
import numpy as np
from sklearn.metrics import r2_score
# Import LGBMRegressor instead of Lasso
from lightgbm import LGBMRegressor 

# Create indices and shuffle once
lst = list(range(len(embeds)))
random.seed(42)
random.shuffle(lst)

# Create multiple random 80/20 splits
n_splits = 5
splits = []

for i in range(n_splits):
    # Reshuffle for each split while maintaining same splits across targets
    temp_lst = lst.copy()
    random.seed(42 + i)  # Different seed for each split
    random.shuffle(temp_lst)
    
    split_point = int(len(temp_lst) * 0.8)
    train_idxs = temp_lst[:split_point]
    val_idxs = temp_lst[split_point:]
    splits.append((train_idxs, val_idxs))

# Convert embeds to numpy array once for efficiency
embeds_np = np.array(torch.cat(embeds))
regressors = [[None for i in range(5)] for j in range(5)]
# Now iterate through each target
for i in range(5):
    print(f"\n=== Target {i+1} ===")
    targets_np = np.array(torch.cat(targets[i]))
    
    split_scores = []
    
    for split_idx, (train_idxs, val_idxs) in enumerate(splits):
        print(f"Fold {split_idx+1}:")
        X_train, y_train = embeds_np[train_idxs], targets_np[train_idxs]
        X_val, y_val = embeds_np[val_idxs], targets_np[val_idxs]
        
        # Replace Lasso() with LGBMRegressor() and set random_state for determinism
        reg = LGBMRegressor(random_state=42)
        # Note: .ravel() is used because LGBMRegressor expects y to be 1-dimensional
        reg.fit(X_train, y_train.ravel()) 
        
        train_preds = reg.predict(X_train)
        train_preds[train_preds < 0.0] = 0.0
        train_r2 = r2_score(y_train, train_preds)
        
        val_preds = reg.predict(X_val)
        val_preds[val_preds < 0.0] = 0.0
        val_r2 = r2_score(y_val, val_preds)
        
        print(f"  Train R²: {train_r2:.4f}")
        print(f"  Val R²: {val_r2:.4f}")
        split_scores.append((train_r2, val_r2))
        regressors[i][split_idx] = reg
    
    # Print summary for this target
    avg_train_r2 = np.mean([score[0] for score in split_scores])
    avg_val_r2 = np.mean([score[1] for score in split_scores])
    print(f"\nTarget {i+1} Average:")
    print(f"  Avg Train R²: {avg_train_r2:.4f}")
    print(f"  Avg Val R²: {avg_val_r2:.4f}")

mapping = {"Dry_Clover_g": 0, "Dry_Dead_g": 1, "Dry_Green_g": 2, "Dry_Total_g": 3, "GDM_g": 4}

test_embeds = {}
counter = 0
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224)), transforms.Normalize(mean, std)])
test_df = pd.read_csv("/kaggle/input/csiro-biomass/test.csv")
root = "/kaggle/input/csiro-biomass/"
sample_ids = []
for i in range(len(test_df)):
    entry = test_df.iloc[i]
    file_path = root + entry['image_path']
    sample_id = entry['sample_id']
    #y = torch.tensor([[entry['target']]])
    if sample_id not in sample_ids:
        img = Image.open(file_path)
        x = torch.tensor(processor(img).pixel_values)
        with torch.no_grad():
            x = x.cuda()
            test_embeds[sample_id.split("_")[0]] = model(x).pooler_output.cpu()
            counter += 1
        sample_ids.append(sample_id)
    if counter % 100 == 0:
        print(f"{counter} batches processed.")


predictions = []
sample_ids = []
test_df = pd.read_csv("/kaggle/input/csiro-biomass/test.csv")
for i in range(len(test_df)):
    try:
        entry = test_df.iloc[i]
        X = np.array(test_embeds[entry['sample_id'].split("__")[0]])
        sample_ids.append(entry['sample_id'])
        models = regressors[mapping[entry['sample_id'].split("__")[1]]]
        prediction = 0.0
        for item in models:
            single_pred = item.predict(X)
            # Clip predictions to non-negative values
            if single_pred < 0.0:
                single_pred = 0.0
            prediction += single_pred
        prediction = prediction / 5
        predictions.append(float(prediction))
    except Exception as e:
        predictions.append(0.0)


%cd /kaggle/working
submission = pd.DataFrame({
    'sample_id': sample_ids,
    'target': predictions
})

submission.to_csv('submission5.csv', index=False)

## Final Ensemble

In [ ]:
sub1 = pd.read_csv('submission.csv') ## W= 0.52189
sub2 = pd.read_csv('submission2.csv') ## W = 0.47811
sub3 = pd.read_csv('submission3.csv') ## W = 0.47811
sub4 = pd.read_csv('submission4.csv')
sub5 = pd.read_csv('submission5.csv')
sub6 = pd.read_csv('submission_convnext_512_Imsize.csv')

sub1['target'] = 0.55*(0.00107*sub1['target'] + 0.34153*sub2['target']+ 0.08650*sub3['target'] + 0.57091*sub6['target']) + 0.45*(0.7*sub4['target'] + 0.3*sub5['target'])
sub1.to_csv('submission.csv', index = False)
sub1.head()

## Final Ensemble